# [Moduel 2.2] AutoGluon-Tabular in AWS Marketplace

본 노트북은 아래와 같은 부분을 작업 합니다.
- 본 노트북은 2.1.AutoGluon-MKP.ipynb 아래를 제외하고 동일 합니다.
    - 실제 클래스의 값(예: 1, 2)를 예측값으로 받는 것이 아니라 확률값(에: 0.24, 0.89)을 받는 예제 입니다. 
    - 위의 버전은 `Latest Version: 3.4` 를 기반으로 하였습니다.
        - 3.4 미만의 버전에서는 작동하지 않습니다.
    - [오토 글루온마켓 플레이스](https://aws.amazon.com/marketplace/pp/prodview-n4zf5pmjt7ism) 에서 확인 하시면 됩니다.

### 이 노트북은 소요 시간이 약 30 분 걸립니다.


### Contents:
* [Step 1: Subscribe to AutoML algorithm from AWS Marketplace](#Step-1:-Subscribe-to-AutoML-algorithm-from-AWS-Marketplace)
* [Step 2: Set up environment](#Step-2-:-Set-up-environment)
* [Step 3: Prepare and upload data](#Step-3:-Prepare-and-upload-data)
* [Step 4: Train a model](#Step-4:-Train-a-model)
* [Step 5: Deploy the model and perform a real-time inference](#Step-5:-Deploy-the-model-and-perform-a-real-time-inference)
* [Step 6: Clean-up](#Step-6:-Clean-up)

### Step 1: Subscribe to AutoML algorithm from AWS Marketplace

1. 마켓플레이스에 접속하고 [AutoGluon-Tabular](https://aws.amazon.com/marketplace/pp/prodview-n4zf5pmjt7ism) 페이지를 오픈합니다.
2. **Highlights** 부분과 **product overview** 부분을 읽어봅니다. (알고리즘의 개요와 동작특성, 특장점 등을 설명하고 있습니다.)
3. **usage information** 부분과 **additional resources** 부분을 살펴봅니다. (알고리즘의 사용방법이 설명됩니다.)
4. 지원되는 인스턴스 타입을 살펴봅니다. 본 노트북의 이후 셀에서 해당 타입을 설정할 것입니다. 
5. **Continue to subscribe** 버튼을 클릭합니다.
6. **End user license agreement**, **support terms**, **pricing information**을 읽어봅니다.
7. 여러분의 조직에서 해당 알고리즘의 라이센스, 가격, 지원정책에 동의하는 경우 **Accept offer** 버튼을 클릭합니다. 

**Notes**: 
1. **Continue to configuration** 버튼이 활성회되면 여러분의 어카운트가 subscription 된 상태입니다. 
2. **Continue to configuration** 버튼을 클릭하고 리전을 선택하면 `Product Arn`을 확인할 수 있습니다. 이 값이 여러분의 학습작업에서 사용할 알고리즘 ARN입니다. (단, 본 노트북에서는 이미 리전별 ARN 값들을 **src/algorithm_arns.py** 파일에 저장해 두었기 때문에 특별히 설정할 필요는 없습니다.) 


### Step 2 : Set up environment

In [1]:
# Import the latest sagemaker and boto3 SDKs
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install -qU awscli boto3 "sagemaker>=2.0.0" tqdm
!{sys.executable} -m pip show sagemaker

Name: sagemaker
Version: 2.24.1
Summary: Open source library for training and deploying models on Amazon SageMaker.
Home-page: https://github.com/aws/sagemaker-python-sdk/
Author: Amazon Web Services
Author-email: None
License: Apache License 2.0
Location: /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages
Requires: smdebug-rulesconfig, google-pasta, protobuf3-to-dict, protobuf, attrs, boto3, importlib-metadata, numpy, packaging
Required-by: 


In [2]:
#Import necessary libraries.
import os
import boto3
import sagemaker
from time import sleep
from collections import Counter
import numpy as np
import pandas as pd
from sagemaker import get_execution_role, local, Model, utils, fw_utils, s3
from sagemaker import AlgorithmEstimator
from sagemaker.predictor import RealTimePredictor, csv_serializer, StringDeserializer
from sklearn.metrics import accuracy_score, classification_report
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell

# Print settings
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 10)

# Account/s3 setup
session = sagemaker.Session()
bucket = session.default_bucket()
prefix = 'sagemaker/autogluon-tabular'
region = session.boto_region_name
role = get_execution_role()


In [3]:
compatible_training_instance_type='ml.m5.4xlarge' 
compatible_inference_instance_type='ml.m5.4xlarge' 

In [4]:
#Specify algorithm ARN for AutoGluon-Tabular from AWS Marketplace.  However, for this notebook, the algorithm ARN 
#has been specified in src/algorithm_arns.py file and you do not need to specify the same explicitly.

from src.algorithm_arns import AlgorithmArnProvider


# algorithm_arn = AlgorithmArnProvider.get_algorithm_arn(region)

#### 마켓 플레이스에서 아래의 이미지 ARN을 가져옵니다

In [5]:

algorithm_arn = 'arn:aws:sagemaker:ap-northeast-2:745090734665:algorithm/autogluon-tabular-v3-4-b164f6c99d3e4ca2b6b56e72df06a4f2'
print(algorithm_arn)

arn:aws:sagemaker:ap-northeast-2:745090734665:algorithm/autogluon-tabular-v3-4-b164f6c99d3e4ca2b6b56e72df06a4f2


### Step 3: Get the data

기존에 만든 데이터를 로딩하고 합니다. AutoGluon 이용시 별도의 검증(validation)셋의 생성은 필요하지 않습니다. (내부적으로 k-fold cross-validation을 이용합니다.)


In [6]:
%store -r no_auto_train_file
%store -r no_auto_test_file

train = pd.read_csv(no_auto_train_file)
test = pd.read_csv(no_auto_test_file)

# Split test X/y
label = 'classes'
y_test = test[label]
X_test = test.drop(columns=[label])
print("auto_train_file: ", train.shape)
print("auto_train_file: ", test.shape)


auto_train_file:  (53740, 18)
auto_train_file:  (13436, 18)


- train, test 데이터는 레이블 컬럼 `classes`를 포함하고 있습니다.
- X_test 데이터는 레이블 컬럼 `classes`를 포함하고 있지 않습니다.



In [7]:
from IPython.display import display as dp
dp(train.head(2))
dp(test.head(2))
dp(X_test.head(2))




,classes,customer_zip_code_prefix,customer_city,customer_state,price,freight_value,product_weight_g,product_category_name_english,seller_zip_code_prefix,seller_city,seller_state,order_weekday,order_day,order_month,customer_seller_state,customer_seller_city,customer_seller_zip_code_prefix,product_volume
0,3,12030,taubate,SP,29.99,10.96,9000.0,fashion_shoes,13481,limeira,SP,1,4,10,SP_SP,taubate_limeira,12030_13481,2640.0
1,2,78075,cuiaba,MT,23.90,26.82,1500.0,furniture_decor,17051,bauru,SP,1,4,10,MT_SP,cuiaba_bauru,78075_17051,14625.0


,classes,customer_zip_code_prefix,customer_city,customer_state,price,freight_value,product_weight_g,product_category_name_english,seller_zip_code_prefix,seller_city,seller_state,order_weekday,order_day,order_month,customer_seller_state,customer_seller_city,customer_seller_zip_code_prefix,product_volume
0,3,89219,joinville,SC,105.00,23.89,1000.0,watches_gifts,21840,rio de janeiro,RJ,1,19,6,SC_RJ,joinville_rio de janeiro,89219_21840,7632.0
1,0,9950,diadema,SP,99.97,15.80,650.0,housewares,32677,betim,MG,1,19,6,SP_MG,diadema_betim,9950_32677,9600.0


,customer_zip_code_prefix,customer_city,customer_state,price,freight_value,product_weight_g,product_category_name_english,seller_zip_code_prefix,seller_city,seller_state,order_weekday,order_day,order_month,customer_seller_state,customer_seller_city,customer_seller_zip_code_prefix,product_volume
0,89219,joinville,SC,105.00,23.89,1000.0,watches_gifts,21840,rio de janeiro,RJ,1,19,6,SC_RJ,joinville_rio de janeiro,89219_21840,7632.0
1,9950,diadema,SP,99.97,15.80,650.0,housewares,32677,betim,MG,1,19,6,SP_MG,diadema_betim,9950_32677,9600.0


데이터를 S3로 업로드합니다. 

In [8]:
from p_utils import upload_s3

# Upload train. test file
data_prefix = 'autogluon/predict-deliver-time/data'

train_s3_path = upload_s3(bucket, no_auto_train_file, data_prefix)
test_s3_path = upload_s3(bucket, no_auto_test_file, data_prefix)
print("train_s3_path: ", train_s3_path)
print("test_s3_path: ", test_s3_path)

train_s3_path:  s3://sagemaker-ap-northeast-2-057716757052/autogluon/predict-deliver-time/data/preproc_data/auto_no_fe/train/train.csv
test_s3_path:  s3://sagemaker-ap-northeast-2-057716757052/autogluon/predict-deliver-time/data/preproc_data/auto_no_fe/test/test.csv


### Step 4: Train a model

이제 모델을 학습하겠습니다. 

**주의:** 적절한 디스크 사이즈 할당을 위해 `train_volume_size`값을 조정해야 할 수 있습니다.


In [9]:
# Define required label and optional additional parameters
fit_args = {
  'label': 'classes',
  # Adding 'best_quality' to presets list will result in better performance (but longer runtime)
  'presets': ['optimize_for_deployment'],
}

# Pass fit_args to SageMaker estimator hyperparameters
hyperparameters = {
  'fit_args': fit_args,
  'feature_importance': True
}

In [10]:
algo = AlgorithmEstimator(algorithm_arn=algorithm_arn, 
                          role=role, 
                          instance_count=1, 
                          instance_type=compatible_training_instance_type, 
                          sagemaker_session=session, 
                          base_job_name='autogluon',
                          hyperparameters=hyperparameters,
                          train_volume_size=100) 

inputs = {'training': train_s3_path, 'testing': test_s3_path}

algo.fit(inputs)

2021-02-03 06:26:28 Starting - Starting the training job...
2021-02-03 06:26:30 Starting - Launching requested ML instancesProfilerReport-1612333587: InProgress
......
2021-02-03 06:27:55 Starting - Preparing the instances for training...
2021-02-03 06:28:15 Downloading - Downloading input data
2021-02-03 06:28:15 Training - Downloading the training image......
2021-02-03 06:29:22 Training - Training image download completed. Training in progress.2021-02-03 06:29:22,213 sagemaker-training-toolkit INFO     Imported framework sagemaker_mxnet_container.training
2021-02-03 06:29:22,215 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-02-03 06:29:22,215 sagemaker-training-toolkit INFO     Failed to parse hyperparameter fit_args value {'label': 'classes', 'presets': ['optimize_for_deployment']} to Json.
Returning the value itself
2021-02-03 06:29:22,215 sagemaker-training-toolkit INFO     Failed to parse hyperparameter feature_importance value True to J

### Step 5: Deploy the model and perform a real-time inference

#### 추론을 위한 직렬화, 비직렬화 세팅
- 직렬화로서 `CSVSerializer` 를 선언하고, 추론으로서 제공하는 MIME type 을 아래와 같이 알려 줍니다.
`CSVSerializer.CONTENT_TYPE = 'text/csv;proba'
    - 'text/csv;proba': 확률값을 예측값으로서 받습니다. (분류의 갯수만큼의 확률값을 받습니다. 예: 0.21, 0.79)
    - 'text/csv' 분류값으로서 예측값을 받습니다. (에: 0, 혹은 1)
- 비직렬화로서 StringDeserializer 를 선언 합니다.


In [11]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import StringDeserializer

# CSVSerializer.ACCEPT = 'text/csv;proba'
CSVSerializer.CONTENT_TYPE = 'text/csv;proba'

### 엔드포인트 생성
위의 직렬화, 비직렬화 오브젝트를 인자로 제공하고 엔드포인트 생성 합니다.

In [12]:
%%time


predictor = algo.deploy(1, 
                        compatible_inference_instance_type, 
                        serializer=CSVSerializer(), 
                        deserializer=StringDeserializer())

..........
-------------!CPU times: user 274 ms, sys: 8.52 ms, total: 283 ms
Wall time: 7min 17s


## 추론 실행

일단 1개의 추론의 입력값 (payload)를 확인하고, 예측을 합니다.
아래와 같은 결과를 도출할 수 있습니다.
```
payload: 
 customer_zip_code_prefix,customer_city,customer_state,price,freight_value,product_weight_g,product_category_name_english,seller_zip_code_prefix,seller_city,seller_state,order_weekday,order_day,order_month,customer_seller_state,customer_seller_city,customer_seller_zip_code_prefix,product_volume
89219,joinville,SC,105.0,23.89,1000.0,watches_gifts,21840,rio de janeiro,RJ,1,19,6,SC_RJ,joinville_rio de janeiro,89219_21840,7632.0

prediction probability:  0.019505123,0.06890744,0.3856581,0.31477204,0.21115737
```

In [14]:
sample = X_test[0:1].to_csv(index=False)
print("payload: \n", sample)
print("prediction probability: ", predictor.predict(sample))

payload: 
 customer_zip_code_prefix,customer_city,customer_state,price,freight_value,product_weight_g,product_category_name_english,seller_zip_code_prefix,seller_city,seller_state,order_weekday,order_day,order_month,customer_seller_state,customer_seller_city,customer_seller_zip_code_prefix,product_volume
89219,joinville,SC,105.0,23.89,1000.0,watches_gifts,21840,rio de janeiro,RJ,1,19,6,SC_RJ,joinville_rio de janeiro,89219_21840,7632.0

prediction probability:  0.019505123,0.06890744,0.3856581,0.31477204,0.21115737



In [39]:
sample = X_test[1:2].to_csv(index=False)
print("payload: \n", sample)
print("prediction probability: ", predictor.predict(sample))

payload: 
 customer_zip_code_prefix,customer_city,customer_state,price,freight_value,product_weight_g,product_category_name_english,seller_zip_code_prefix,seller_city,seller_state,order_weekday,order_day,order_month,customer_seller_state,customer_seller_city,customer_seller_zip_code_prefix,product_volume
9950,diadema,SP,99.97,15.8,650.0,housewares,32677,betim,MG,1,19,6,SP_MG,diadema_betim,9950_32677,9600.0

prediction probability:  0.042804297,0.23574851,0.49924144,0.14277346,0.07943225



#### 예측 확률값을 클래스 분류 값으로 변환

In [66]:
import numpy as np
prob_result = predictor.predict(sample)
def judge_class_lable(prob_result):
    '''
    예측값을 받고 파싱하여, 클래스 레이블값과 확률값을 제공합니다.
    확률값을 가지고 Threshold 를 정하여 (예: 0.8) 최종 례이블 값을 정할 수도 있습니다.
    '''
    # print(prob_result)
    prob_result = prob_result.replace('\n','')
    prob_result = prob_result.split(',')
    new_list = []
    for item in prob_result:
        new_list.append(float(item))    
    # print(new_list)
    i = np.argmax(new_list, axis=0)

    return i, new_list[i]
    
    
class_label, class_prob = judge_class_lable(prob_result)    
print("class label, probability: \n", class_label, "\t", class_prob)

class label, probability: 
 2 	 0.49924144


### Step 6: Clean-up

예측작업이 끝나면 추가 과금을 피하기 위해 엔드포인트를 삭제합니다. 


In [ ]:
predictor.delete_endpoint()

In [ ]:
#Finally, delete the model you created.
# predictor.delete_model()

마지막으로, 테스트만을 목적으로 AWS 마켓플레이스에 subscribe한 경우 테스트 이후 unsubscribe를 할 수 있습니다.  
subscription을 취소하기 전에 해당 알고리즘이나 모델 패키지로부터 배포된 [모델](https://console.aws.amazon.com/sagemaker/home#/models)에 있지 않은지 확인합니다. - 모델과 연관된 컨테이너를 통해 이를 확인할 수 있습니다.

AWS 마켓플레이스 unsubscribe 하기
1. [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=lbr_tab_ml)의 __Machine Learning__ 탭으로 이동합니다.
1. subscription을 취소하고자 하는 리스트로 이동한 후 __Cancel Subscription__을 클릭합니다.
